In [1]:
from langchain.tools.retriever import create_retriever_tool

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
import os

from tqdm.auto import tqdm

In [6]:
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = os.getenv('MILVUS_USER')
MILVUS_PASSWORD = os.getenv('MILVUS_PASSWORD')
MILVUS_COLLECTION = "demo_collection"

In [8]:
model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(
    #model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=False
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

/opt/app-root/lib64/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
retriever = store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4}
            )

In [10]:
rag_tool_description = """
Use this tool when seaching for information about Red Hat OpenShift AI
"""

In [11]:
tool_retriever = create_retriever_tool(retriever, "RAG_tool", rag_tool_description)

In [12]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"